<a href="https://colab.research.google.com/github/Abhishek2104200/Wafer-Defects/blob/main/rp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision transformers timm matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pickle

# Load the dataset
with open('/content/balanced.pkl', 'rb') as f:
    data = pickle.load(f)

# Print available keys
print("Dataset Keys:", data.keys())

# Extract wafer images
X = data['waferMap']  # Wafer maps are your image data
y = data['failureNum']  # Assuming 'failureNum' is the defect label

# Check shapes
print(f"Number of images: {len(X)}, Labels: {len(y)}")


Dataset Keys: Index(['waferMap', 'dieSize', 'lotName', 'trainTestLabel', 'failureType',
       'waferMapDim', 'failureNum', 'trainTestNum'],
      dtype='object')
Number of images: 4000, Labels: 4000


In [ ]:
for i in range(5):  # Check first 5 samples
    print(f"Sample {i} type: {type(X[i])}, Shape: {np.array(X[i]).shape if X[i] is not None else 'None'}")


Sample 0 type: <class 'numpy.ndarray'>, Shape: (41, 33)
Sample 1 type: <class 'numpy.ndarray'>, Shape: (44, 41)
Sample 2 type: <class 'numpy.ndarray'>, Shape: (25, 27)
Sample 3 type: <class 'numpy.ndarray'>, Shape: (25, 27)
Sample 4 type: <class 'numpy.ndarray'>, Shape: (31, 31)


In [ ]:
import cv2

filtered_X = []
filtered_y = []

for i in range(len(X)):
    if X[i] is not None and isinstance(X[i], np.ndarray):  # Only keep valid arrays
        img = cv2.resize(X[i], (64, 64))  # Resize to 64x64
        filtered_X.append(img)
        filtered_y.append(y[i])

# Convert to NumPy arrays
X_resized = np.array(filtered_X, dtype=np.float32) / 255.0  # Normalize
y_resized = np.array(filtered_y, dtype=np.int64)

print(f"Final dataset size: {X_resized.shape}, Labels: {y_resized.shape}")


Final dataset size: (4000, 64, 64), Labels: (4000,)


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Add channel dimension (1 for grayscale)
X_tensor = torch.tensor(X_resized).unsqueeze(1)  # Shape: (4000, 1, 64, 64)
y_tensor = torch.tensor(y_resized)

# Train-Test Split (80-20)
split = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:split], X_tensor[split:]
y_train, y_test = y_tensor[:split], y_tensor[split:]

# Create DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")


Train size: 3200, Test size: 800


In [ ]:
import torch.nn as nn
import torch.optim as optim

class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 64x64 → 32x32
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2)   # 32x32 → 16x16
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize model, optimizer, and loss function
dae = DenoisingAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(dae.parameters(), lr=0.001)

# Train Denoising Autoencoder
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, _ in train_loader:
        noisy_inputs = inputs + 0.1 * torch.randn_like(inputs)  # Add noise
        noisy_inputs = torch.clamp(noisy_inputs, 0., 1.)

        optimizer.zero_grad()
        outputs = dae(noisy_inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.09552264208556153
Epoch 2/10, Loss: 0.00021405488339951262
Epoch 3/10, Loss: 2.4557202559662984e-05
Epoch 4/10, Loss: 1.583666482474655e-05
Epoch 5/10, Loss: 1.330331620920333e-05
Epoch 6/10, Loss: 1.2265159602975472e-05
Epoch 7/10, Loss: 1.1759212675315212e-05
Epoch 8/10, Loss: 1.1487956662676879e-05
Epoch 9/10, Loss: 1.124259350945067e-05
Epoch 10/10, Loss: 1.1034027666028124e-05


In [ ]:
dae.eval()  # Set autoencoder to evaluation mode

X_denoised = []

with torch.no_grad():
    for img in X_tensor:
        img = img.unsqueeze(0)  # Add batch dimension
        denoised_img = dae(img)
        X_denoised.append(denoised_img.squeeze(0).numpy())  # Remove batch dim

X_denoised = np.array(X_denoised)  # Convert to numpy array

print(f"Denoised images shape: {X_denoised.shape}")  # Should be (4000, 1, 64, 64)


Denoised images shape: (4000, 1, 64, 64)


In [ ]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X_denoised, dtype=torch.float32)
y_tensor = torch.tensor(y_resized, dtype=torch.long)

# Train-Test Split (80-20)
split = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:split], X_tensor[split:]
y_train, y_test = y_tensor[:split], y_tensor[split:]

# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

print(f"Final Training Size: {len(X_train)}, Testing Size: {len(X_test)}")


Final Training Size: 3200, Testing Size: 800


# **MODEL TRAINING**

In [ ]:
import torch.nn as nn
from torchvision import models
from transformers import ViTModel, ViTConfig

# CNN Backbone (MobileNetV2)
cnn_model = models.mobilenet_v2(pretrained=True)
cnn_model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Adjust for grayscale input

# Vision Transformer (ViT)
vit_config = ViTConfig(image_size=64, num_channels=1, num_labels=len(set(y_resized)), hidden_size=768)
vit_model = ViTModel(vit_config)

# Hybrid Model
class HybridCNNViT(nn.Module):
    def __init__(self, cnn_model, vit_model, num_classes):
        super(HybridCNNViT, self).__init__()
        self.cnn = cnn_model.features
        self.vit = vit_model
        self.fc = nn.Linear(vit_config.hidden_size, num_classes)

    def forward(self, x):
        cnn_features = self.cnn(x)
        cnn_features = cnn_features.view(cnn_features.size(0), -1)
        vit_output = self.vit(pixel_values=x).last_hidden_state[:, 0, :]
        combined_features = cnn_features + vit_output
        output = self.fc(combined_features)
        return output

# Initialize the model
model = HybridCNNViT(cnn_model, vit_model, num_classes=len(set(y_resized)))

print("Hybrid model initialized successfully!")


Hybrid model initialized successfully!


In [ ]:
import torch.nn as nn

class HybridCNNViT(nn.Module):
    def __init__(self, cnn_model, vit_model, num_classes):
        super(HybridCNNViT, self).__init__()
        self.cnn = cnn_model.features  # Extract CNN features
        self.vit = vit_model  # Extract ViT features

        # Reduce CNN output size to 768 using a linear layer
        self.cnn_fc = nn.Linear(20480, 768)

        # Fully Connected Layer for Classification
        self.fc = nn.Linear(768, num_classes)

    def forward(self, x):
        # CNN Feature Extraction
        cnn_features = self.cnn(x)
        cnn_features = cnn_features.view(cnn_features.size(0), -1)  # Flatten
        cnn_features = self.cnn_fc(cnn_features)  # Reduce to 768

        # ViT Feature Extraction
        vit_output = self.vit(pixel_values=x).last_hidden_state[:, 0, :]  # CLS token output

        # Combine CNN & ViT Features
        combined_features = cnn_features + vit_output
        output = self.fc(combined_features)  # Classification

        return output

# Initialize the model
num_classes = len(set(y_resized))
model = HybridCNNViT(cnn_model, vit_model, num_classes)

print("Fixed Hybrid CNN + ViT Model Ready!")


Fixed Hybrid CNN + ViT Model Ready!


In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU if available

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")


Epoch 1/20, Loss: 1.5216, Train Acc: 65.53%
Epoch 2/20, Loss: 0.6730, Train Acc: 76.72%
Epoch 3/20, Loss: 0.4850, Train Acc: 82.91%
Epoch 4/20, Loss: 0.4605, Train Acc: 84.03%
Epoch 5/20, Loss: 0.3488, Train Acc: 87.47%
Epoch 6/20, Loss: 0.3763, Train Acc: 87.62%
Epoch 7/20, Loss: 0.3058, Train Acc: 89.81%
Epoch 8/20, Loss: 0.2655, Train Acc: 91.03%
Epoch 9/20, Loss: 0.2747, Train Acc: 90.50%
Epoch 10/20, Loss: 0.2544, Train Acc: 91.50%
Epoch 11/20, Loss: 0.2268, Train Acc: 92.56%
Epoch 12/20, Loss: 0.2134, Train Acc: 92.69%
Epoch 13/20, Loss: 0.2423, Train Acc: 92.56%
Epoch 14/20, Loss: 0.2132, Train Acc: 92.97%
Epoch 15/20, Loss: 0.1997, Train Acc: 93.84%
Epoch 16/20, Loss: 0.1942, Train Acc: 93.56%
Epoch 17/20, Loss: 0.1607, Train Acc: 95.09%
Epoch 18/20, Loss: 0.1590, Train Acc: 95.38%
Epoch 19/20, Loss: 0.1333, Train Acc: 95.91%
Epoch 20/20, Loss: 0.1590, Train Acc: 95.09%


# **EVALUATION**

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 10.25%


# **SAVING THE MODEL**

In [ ]:
torch.save(model.state_dict(), "hybrid_cnn_vit.pth")
print("Model saved successfully!")


Model saved successfully!


# **VALIDATION**

In [ ]:
num_classes = len(set(y_resized))
print("Number of classes:", num_classes)


Number of classes: 8


In [ ]:
print("Model output size:", model.fc.out_features)  # Should match num_classes


Model output size: 8


In [ ]:
print("Unique Train Labels:", np.unique(y_train))
print("Unique Test Labels:", np.unique(y_test))


Unique Train Labels: [0 1 2 3 4 5 6 7]
Unique Test Labels: [0 1 2 3 4 5 6 7]


In [ ]:
# Correct label mapping BEFORE splitting
label_mapping = {old_label: new_label for new_label, old_label in enumerate(sorted(set(y_resized)))}
y_resized = np.array([label_mapping[label] for label in y_resized])  # Apply mapping correctly

# Verify again
print("Updated Unique Labels:", np.unique(y_resized))  # Should be [0 1 2 3 4 5 6 7]


Updated Unique Labels: [0 1 2 3 4 5 6 7]


In [ ]:
from sklearn.model_selection import train_test_split

# Ensure the split is done AFTER fixing labels
X_train, X_test, y_train, y_test = train_test_split(X_denoised, y_resized, test_size=0.2, stratify=y_resized, random_state=42)

# Verify labels again
print("Fixed Unique Train Labels:", np.unique(y_train))  # Should be [0 1 2 3 4 5 6 7]
print("Fixed Unique Test Labels:", np.unique(y_test))  # Should be [0 1 2 3 4 5 6 7]


Fixed Unique Train Labels: [0 1 2 3 4 5 6 7]
Fixed Unique Test Labels: [0 1 2 3 4 5 6 7]


In [ ]:
print("Model Output Size:", model.fc.out_features)  # Should be 8


Model Output Size: 8
